## Minimize Net System Carbon Forest Emission (Loop to Model Each Scenario with HWP Half-life Increasing From 1 to 200 Years)

This script was also run 25 times under scenarios with HWP displacement factor increasing from 0.0 to 2.4. So, 5,000 scenarios with different combinations of HWP half-life and displacement factor were run and saved.

### Set Up Modelling Environment and Parameters

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Install libcbm Module from GitHub and Import it
install_libcbm = False
if install_libcbm:
    %pip install -U git+https://github.com/cat-cfs/libcbm_py.git@main
    import libcbm
    libcbm.__path__

In [3]:
# Import Modules
import matplotlib.pyplot as plt
import pandas as pd
import ws3.forest, ws3.core
import math
import libcbm
import os

In [4]:
#Define constants from the product carbon estimation
# Define the product half-lives（>0.69)
half_life_paper = 2  # default value
# Define the allocation distribution
proportion_solid_wood = 1.0 # Assume all harvested wood is made into a single type of HWP to simplify the experiment
# Define the utilization rate
util_rate = 0.85
# Define the HWPs displacement factor
displacement_factor_solid = 2.2 # default value
# Define the Credibility
credit =  1.0 # default value

In [5]:
from util import compile_scenario, plot_scenario, plugin_c_curves, cmp_c_z, cmp_c_caa, cmp_c_ci

In [6]:
#Install Module
%pip install gurobipy
#Import Module
import gurobipy as grb

In [7]:
# Ensure the results directories exist
base_dir = f'results/minimize_emission_loop/half_life/even_flow/df_{displacement_factor_solid}' #Choose the file folder where you save results of each scenario
harvest_dir = os.path.join(base_dir, 'harvest')
stock_dir = os.path.join(base_dir, 'stock')
emission_dir = os.path.join(base_dir, 'emission')

In [8]:
iteration = 200

In [9]:
# Define Disturance Types
disturbance_type_mapping = [{'user_dist_type': 'harvest', 'default_dist_type': 'Clearcut harvesting without salvage'},
                            {'user_dist_type': 'fire', 'default_dist_type': 'Wildfire'}]

In [10]:
# Define Carbon Pools
biomass_pools = ['SoftwoodMerch','SoftwoodFoliage', 'SoftwoodOther', 'SoftwoodCoarseRoots','SoftwoodFineRoots',                        
                 'HardwoodMerch', 'HardwoodFoliage', 'HardwoodOther', 'HardwoodCoarseRoots', 'HardwoodFineRoots']
dom_pools = ['AboveGroundVeryFastSoil', 'BelowGroundVeryFastSoil', 'AboveGroundFastSoil', 'BelowGroundFastSoil',
             'MediumSoil', 'AboveGroundSlowSoil', 'BelowGroundSlowSoil', 'SoftwoodStemSnag', 'SoftwoodBranchSnag',
             'HardwoodStemSnag', 'HardwoodBranchSnag']
emissions_pools = ['CO2', 'CH4', 'CO', 'NO2']
products_pools = ['Products']
ecosystem_pools = biomass_pools + dom_pools
all_pools = biomass_pools + dom_pools + emissions_pools + products_pools

# Define Carbon Fluxes
annual_process_fluxes = [
    'DecayDOMCO2Emission',
    'DeltaBiomass_AG',
    'DeltaBiomass_BG',
    'TurnoverMerchLitterInput',
    'TurnoverFolLitterInput',
    'TurnoverOthLitterInput',
    'TurnoverCoarseLitterInput',
    'TurnoverFineLitterInput',
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir'
]

npp_fluxes=[
    'DeltaBiomass_AG', 
    'DeltaBiomass_BG'
]
decay_emissions_fluxes = [
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir'
]

disturbance_production_fluxes = [
    'DisturbanceSoftProduction',
    'DisturbanceHardProduction',
    'DisturbanceDOMProduction'   
]

disturbance_emissions_fluxes = [
    'DisturbanceMerchToAir',
    'DisturbanceFolToAir',
    'DisturbanceOthToAir',
    'DisturbanceCoarseToAir',
    'DisturbanceFineToAir',
    'DisturbanceVFastAGToAir',
    'DisturbanceVFastBGToAir',
    'DisturbanceFastAGToAir',
    'DisturbanceFastBGToAir',
    'DisturbanceMediumToAir',
    'DisturbanceSlowAGToAir',
    'DisturbanceSlowBGToAir',
    'DisturbanceSWStemSnagToAir',
    'DisturbanceSWBranchSnagToAir',
    'DisturbanceHWStemSnagToAir',
    'DisturbanceHWBranchSnagToAir'   
]

all_fluxes = [
    'DisturbanceCO2Production',
    'DisturbanceCH4Production',
    'DisturbanceCOProduction',
    'DisturbanceBioCO2Emission',
    'DisturbanceBioCH4Emission',
    'DisturbanceBioCOEmission',
    'DecayDOMCO2Emission',
    'DisturbanceSoftProduction',
    'DisturbanceHardProduction',
    'DisturbanceDOMProduction',
    'DeltaBiomass_AG',
    'DeltaBiomass_BG',
    'TurnoverMerchLitterInput',
    'TurnoverFolLitterInput',
    'TurnoverOthLitterInput',
    'TurnoverCoarseLitterInput',
    'TurnoverFineLitterInput',
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir',
    'DisturbanceMerchToAir',
    'DisturbanceFolToAir',
    'DisturbanceOthToAir',
    'DisturbanceCoarseToAir',
    'DisturbanceFineToAir',
    'DisturbanceDOMCO2Emission',
    'DisturbanceDOMCH4Emission',
    'DisturbanceDOMCOEmission',
    'DisturbanceMerchLitterInput',
    'DisturbanceFolLitterInput',
    'DisturbanceOthLitterInput',
    'DisturbanceCoarseLitterInput',
    'DisturbanceFineLitterInput',
    'DisturbanceVFastAGToAir',
    'DisturbanceVFastBGToAir',
    'DisturbanceFastAGToAir',
    'DisturbanceFastBGToAir',
    'DisturbanceMediumToAir',
    'DisturbanceSlowAGToAir',
    'DisturbanceSlowBGToAir',
    'DisturbanceSWStemSnagToAir',
    'DisturbanceSWBranchSnagToAir',
    'DisturbanceHWStemSnagToAir',
    'DisturbanceHWBranchSnagToAir'
]

grossgrowth_ag = [
    "DeltaBiomass_AG",
    "TurnoverMerchLitterInput",
    "TurnoverFolLitterInput",
    "TurnoverOthLitterInput",
]

grossgrowth_bg = [
    "DeltaBiomass_BG",
    "TurnoverCoarseLitterInput",
    "TurnoverFineLitterInput",
]

product_flux = [
     "DisturbanceSoftProduction",
     "DisturbanceHardProduction",
     "DisturbanceDOMProduction",
]

In [11]:
# Define Sum Carbon Pools and Sum Carbon Fluxes
total_emission = decay_emissions_fluxes + disturbance_emissions_fluxes
gross_growth = grossgrowth_ag + grossgrowth_bg

sum_pools = ['ecosystem', 'biomass', 'DOM']
sum_fluxes = ['total_emission', 'gross_growth', 'net_emission']

### Initialize Forest Model and Insert Carbon Yield Curves

In [12]:
base_year = 2020
horizon = 20
period_length = 10
max_age = 1000
tvy_name = 'totvol'

In [13]:
fm = ws3.forest.ForestModel(model_name='tsa24',
                            model_path='data/woodstock_model_files_tsa24',
                            base_year=base_year,
                            horizon=horizon,
                            period_length=period_length,
                            max_age=max_age)

fm.import_landscape_section()
fm.import_areas_section()
fm.import_yields_section()
fm.import_actions_section()
fm.import_transitions_section()
fm.initialize_areas()
fm.add_null_action()
fm.reset_actions()
fm.grow()

In [14]:
# Flag 'harvest' as a harvesting action in the ws3 model
harvest_acode = 'harvest'
fm.actions[harvest_acode].is_harvest = True

Pickle files for carbon yield cures are generated from the script: generate_c_curves_tsa24.ipynb

In [15]:
# Plug-in Carbon Yield Curves
c_curves_p = pd.read_pickle("curves/c_curves_p_700_age1_new.pkl")
c_curves_f = pd.read_pickle("curves/c_curves_f_700_age1_new.pkl")
plugin_c_curves(fm, c_curves_p, c_curves_f, sum_pools, sum_fluxes)

found match for mask ('?', '?', '2401000', '?', '2401000')
found match for mask ('?', '?', '2401000', '?', '2401000')
found match for mask ('?', '?', '2401001', '?', '2401001')
found match for mask ('?', '?', '2401001', '?', '2401001')
found match for mask ('?', '?', '2401003', '?', '2401003')
found match for mask ('?', '?', '2401003', '?', '2401003')
found match for mask ('?', '?', '2401004', '?', '2401004')
found match for mask ('?', '?', '2401004', '?', '2401004')
found match for mask ('?', '?', '2401005', '?', '2401005')
found match for mask ('?', '?', '2401005', '?', '2401005')
found match for mask ('?', '?', '2401006', '?', '2401006')
found match for mask ('?', '?', '2401006', '?', '2401006')
found match for mask ('?', '?', '2402001', '?', '2402001')
found match for mask ('?', '?', '2402001', '?', '2402001')
found match for mask ('?', '?', '2402005', '?', '2402005')
found match for mask ('?', '?', '2402005', '?', '2402005')
found match for mask ('?', '?', '2402006', '?', '2402006

### Build and Run the Loop

In [16]:
from util import track_system_stock, track_system_emission

The gen_scenario, run_scenario, and cmp_c_ce functions are not imported from the util file but are directly built in the script again to ensure the loop can run successfully

In [17]:
def gen_scenario(fm, name='base', util_rate=1, harvest_acode='harvest',
                 cflw_ha={}, cflw_hv={}, 
                 cgen_ha={}, cgen_hv={}, 
                 cgen_gs={}, tvy_name='totvol', cp_name='ecosystem', cf_name='total_emissions', obj_mode='max', mask=None):
    
    from functools import partial
    import numpy as np
    coeff_funcs = {}
    cflw_e = {}
    cgen_data = {}
    acodes = ['null', harvest_acode] # define list of action codes
    vexpr = '%s * %0.2f' % (tvy_name, util_rate) # define volume expression

    #Define constants from product carbon estimation

    if obj_mode == 'max': # maximize harvest volume
        sense = ws3.opt.SENSE_MAXIMIZE 
        zexpr = vexpr
    elif obj_mode == 'min': # maximize harvest volume
        sense = ws3.opt.SENSE_MINIMIZE 
        zexpr = vexpr
    else:
        raise ValueError('Invalid obj_mode: %s' % obj_mode)
    
    coeff_funcs['z'] = partial(cmp_c_ce, expr=vexpr) # define objective function coefficient function for net system carbon emission
    
    T = fm.periods
    if cflw_ha: # define even flow constraint (on harvest area)
        cname = 'cflw_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_ha
    if cflw_hv: # define even flow constraint (on harvest volume)
        cname = 'cflw_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_hv         
    if cgen_ha: # define general constraint (harvest area)
        cname = 'cgen_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_ha
    if cgen_hv: # define general constraint (harvest volume)
        cname = 'cgen_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_hv
    if cgen_gs: # define general constraint (growing stock)
        cname = 'cgen_gs'
        coeff_funcs[cname] = partial(cmp_c_ci, yname=tvy_name, mask=None)
        cgen_data[cname] = cgen_gs

    return fm.add_problem(name, coeff_funcs, cflw_e, cgen_data=cgen_data, acodes=acodes, sense=sense, mask=mask)

In [18]:
def run_scenario(fm, scenario_name='base'):
    #Import Module
    import gurobipy as grb
    
    cflw_ha = {}
    cflw_hv = {}
    cgen_ha = {}
    cgen_hv = {}
    cgen_gs = {}
    # cgen_cp = {}
    # cgen_cf = {}
    
    # define harvest area and harvest volume even-flow constraints
    cflw_ha = ({p:0.05 for p in fm.periods}, 1)
    cflw_hv = ({p:0.05 for p in fm.periods}, 1)

    if scenario_name == 'base': 
        # Base scenario
        print('running base scenario')
    else:
        assert False # bad scenario name
      
    
    p = gen_scenario(fm=fm, 
                     name=scenario_name, 
                     cflw_ha=cflw_ha, 
                     cflw_hv=cflw_hv,
                     cgen_ha=cgen_ha,
                     cgen_hv=cgen_hv,
                     cgen_gs=cgen_gs,)

    p.solver('gurobi')

    fm.reset()
    p.solve()

    if p.status() != ws3.opt.STATUS_OPTIMAL:
        print('Model not optimal.')
        df = None   
    else:
        sch = fm.compile_schedule(p)
        fm.apply_schedule(sch, 
                        force_integral_area=False, 
                        override_operability=False,
                        fuzzy_age=False,
                        recourse_enabled=False,
                        verbose=False,
                        compile_c_ycomps=True)
        
        df = compile_scenario(fm)
        fig, ax = plot_scenario(df)
    
    return df, fig, ax

In [19]:
def run_scenario_and_save(fm, half_life_solid_wood):
    # Minimize Forest System Carbon Emission
    print(f'half_life_solid_wood = {half_life_solid_wood}')
    fm.add_null_action()
    p = run_scenario(fm, 'base')

    # Extract results
    df_harvest = compile_scenario(fm)
    df_stock = track_system_stock(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util_rate)
    df_emission = track_system_emission(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util_rate, displacement_factor_solid, credit)
    
    # Save results with filenames using half_life_solid_wood
    df_harvest.to_csv(os.path.join(harvest_dir, f'{half_life_solid_wood}.csv'), index=False)
    df_stock.to_csv(os.path.join(stock_dir, f'{half_life_solid_wood}.csv'), index=False)
    df_emission.to_csv(os.path.join(emission_dir, f'{half_life_solid_wood}.csv'), index=False)

In [ ]:
# Main loop over half_life_solid_wood from 1 to iteration time
for half_life_solid_wood in range(1, iteration+1):

    def cmp_c_ce(fm, path, expr, half_life_solid_wood=half_life_solid_wood, half_life_paper=half_life_paper, proportion_solid_wood=proportion_solid_wood, util_rate=util_rate, displacement_factor_solid=displacement_factor_solid, credit=credit, mask=None):
        
        """
        Compile objective function coefficient for net system carbon emission indicators 
        (given ForestModel instance, leaf-to-root-node path, and expression to evaluate).
        """
        
        hwps_solid_pool = 0.
        hwps_paper_pool = 0.
        hwps_residue_pool = 0.
        hwps_solid_emission = 0.
        hwps_paper_emission = 0.
        hwps_residue_emission = 0.
        result = 0.
        
        # Calculate decay rates based on half-lives
        k_solid_wood = math.log(2) / half_life_solid_wood  # Decay rate for solid wood products (30-year half-life)
        k_paper = math.log(2) / half_life_paper  # Decay rate for paper (2-year half-life)
        
        # Define the allocation distribution
        proportion_paper = 1 - proportion_solid_wood
        
        # wood density (Kennedy, 1965)
        wood_density = 460
    
        # carbon content
        carbon_content = 0.5
        
        for t, n in enumerate(path, start=1):
            
            d = n.data()
    
            eco_emission = fm.inventory(t, 'net_emission', age=d['_age'], dtype_keys=[d['_dtk']])
            
            # Track the new product stock
            if fm.is_harvest(d['acode']):             
                # Calculate new product stock
                new_product_stock = fm.compile_product(t, expr, d['acode'], [d['dtk']], d['age'], coeff=False) * wood_density * carbon_content / 1000  # Convert kg to ton
            else:
                new_product_stock = 0.
            
            # Calculate product stock
            hwps_solid_pool = hwps_solid_pool * (1 - k_solid_wood)**10 + new_product_stock * util_rate * proportion_solid_wood
            hwps_paper_pool = hwps_paper_pool * (1 - k_paper)**10 + new_product_stock * util_rate * proportion_paper
            hwps_residue_pool = new_product_stock * (1.0 - util_rate)
    
            # Calculate prodcut emission
            hwps_solid_emission = hwps_solid_pool * (1- (1 - k_solid_wood)**10)
            hwps_paper_emission = hwps_paper_pool * (1- (1 - k_paper)**10)
            hwps_residue_emission = hwps_residue_pool
            hwps_sum_emission = hwps_solid_emission + hwps_paper_emission + hwps_residue_emission
           
            # Calculate Substitution Effect
            substitution_effect_solid = new_product_stock * util_rate * proportion_solid_wood * displacement_factor_solid * credit  # Emissions avoided by using HWPs
            substitution_effect_paper = 0.
            substitution_effect_residue = 0.
            substitution_effect_sum = substitution_effect_solid + substitution_effect_paper + substitution_effect_residue
            
            # Accumlate the total system carbon stock in each timestep
            result += (eco_emission + hwps_sum_emission - substitution_effect_sum) * -1 * 44 / 12
            # result += (eco_emission + hwps_sum_emission - substitution_effect_sum)
        
        return result
        
    # Run the scenario and save results
    run_scenario_and_save(fm, half_life_solid_wood)